In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import spell
import joblib
import nltk

In [35]:
#Importing and testing custom spell checker based on Norvig and Contraction Expansion
from NorvigSpellCheck import sentenceCorrection
from acronymsExpansion import  expandContractions
sentenceCorrection(expandContractions("is SOP for ntwrk errpr available"))

'is standard operating procedure for network error available'

In [36]:
#Load required Datasets
dataset1=pd.read_csv("dataset1.csv")
dataset2=pd.read_csv("dataset2.csv")

In [37]:
dataset=dataset2.copy()

In [38]:
#Text preprocessing scripts suitable for BPE training
import string
def bpe_clean(s):
#    s=sentenceCorrection(expandContractions(s))
    s=str(s).lower()
    for punctuation in string.punctuation:
        s=s.replace(punctuation," ")
    for n in "123456789":
        s=s.replace(n,"0")
    s=s.lower().split()
    s=[n for n in s if n.isalnum()==True]
    #s=[spell.correction(w) if w.isalpha() else w for w in s]
    s=" ".join(s)
    s=s.encode("ascii","ignore").decode("ascii")
    return s

In [39]:
bpe_clean("Can somebody share error records for FB10' mismatches??")

'can somebody share error records for fb00 mismatches'

In [40]:
#upsample minority classes
from sklearn.utils import resample
def Resample(X_train,y_train):
    dfR=pd.DataFrame(X_train)
    dfR["assignment_group"]=y_train
    sampleData=dfR.groupby("assignment_group").description.count().reset_index().sort_values("description",ascending=True)
    majority_class=sampleData.assignment_group.values[-1]
    minority_classes=list(sampleData.assignment_group.unique())
    minority_classes.remove(majority_class)
    minority_classes

    dfResampled=dfR[dfR.assignment_group==majority_class]
    n_samples=len(dfResampled)
    for minority_class in minority_classes:
#        print(minority_class)
        dfMinority=dfR[dfR.assignment_group==minority_class]
        dfMinorityUpSample= resample(dfMinority,replace=True,random_state=42,n_samples=n_samples)
        dfResampled=pd.concat([dfResampled,dfMinorityUpSample])
        dfResampled.reset_index(inplace=True,drop=True)
    X_train=dfResampled.description
    y_train=dfResampled.assignment_group
    return X_train,y_train
    print(dfResampled.assignment_group.unique())
    print(len(dfResampled))


In [41]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [42]:
#Train Test Split
df2=dataset.copy()
df2["description"]=df2.apply(lambda x: bpe_clean(x.description),axis=1)
X_train, X_test, y_train, y_test = train_test_split(df2.description, df2.assignment_group, test_size=0.20, random_state=42)#,stratify=df.assignment_group)
X_train,y_train=Resample(X_train,y_train)

In [43]:
#Sentence vector for pre-trained BPE
from bpemb import BPEmb

def loadBPE():
    bpemb_en = BPEmb(lang="en", dim=100)
    return bpemb_en

def sent_vec_bpe(sent,bpemb_en):
    wv_res = np.zeros(bpemb_en.emb.vector_size)
    ctr = 1
    for w in bpemb_en.encode(sent):
        if w in bpemb_en.emb:
            ctr += 1
            wv_res += bpemb_en.emb[w]
    wv_res = wv_res/ctr
    #return (wv_res, ctr)
    return wv_res

def get_doc_vecs_bpe(X_train,X_test):
    bpemb_en=loadBPE()
    train_doc_vecs_bpe = []
    for doc in X_train:    
        doc_words = doc
        train_doc_vecs_bpe.append(sent_vec_bpe(doc_words,bpemb_en))


    test_doc_vecs_bpe = []
    for doc in X_test:    
        doc_words = doc
        test_doc_vecs_bpe.append(sent_vec_bpe(doc_words,bpemb_en))
    
    return train_doc_vecs_bpe,test_doc_vecs_bpe


In [1]:
#bpemb_en=loadBPE()
#joblib.dump(bpemb_en,'bpemb_en.pkl')

In [45]:
#Creating Domain Trained BPE
from piecelearn import train
train_text=pd.concat([X_train,X_test])
train_text.to_csv("file.txt",index=False,encoding="utf-8")

my_corpus = "file.txt"
num_pieces = 10000

spm_kwargs = {}
w2v_kwargs = {"sg": 1, "window": 15}

train(my_corpus,
      "spm_model_name",
      num_pieces,
      "w2v_path.txt",
      spm_kwargs,
      w2v_kwargs)

from bpemb.util import sentencepiece_load, load_word2vec_file

bpemb_en_domain = BPEmb(lang="en")
bpemb_en_domain.spm = sentencepiece_load("spm_model_name.model")
bpemb_en_domain.emb = load_word2vec_file("w2v_path.txt")
joblib.dump(bpemb_en_domain,'bpemb_en_domain.pkl')
#s = bpemb_en_domain.encode("the dog flew over the fence")
#print(s)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


10000 100


['bpemb_en_domain.pkl']

In [46]:
#Creation of sentence vector from domain trained BPE
def sent_vec_bpe_domain(sent,bpemb_en_domain):
    wv_res = np.zeros(bpemb_en_domain.emb.vector_size)
    ctr = 1
    for w in bpemb_en_domain.encode(sent):
        if w in bpemb_en_domain.emb:
            ctr += 1
            wv_res += bpemb_en_domain.emb[w]
    wv_res = wv_res/ctr
    #return (wv_res, ctr)
    return wv_res

def get_doc_vecs_bpe_domain(X_train,X_test):
    bpemb_en_domain=joblib.load("bpemb_en_domain.pkl")
    train_doc_vecs_bpe_domain = []
    for doc in X_train:    
        doc_words = doc
        train_doc_vecs_bpe_domain.append(sent_vec_bpe_domain(doc_words,bpemb_en_domain))


    test_doc_vecs_bpe_domain = []
    for doc in X_test:    
        doc_words = doc
        test_doc_vecs_bpe_domain.append(sent_vec_bpe_domain(doc_words,bpemb_en_domain))
        
    return train_doc_vecs_bpe_domain,test_doc_vecs_bpe_domain

In [47]:
#Creation of sentence vector from Glove
from gensim.models.keyedvectors import KeyedVectors
#del glove_model
def loadGlove():
    glove_model = KeyedVectors.load_word2vec_format("glove.6B.100d.w2vformat.txt", binary=False)
    return glove_model


def sent_vec_glove(sent,glove_model):
    wv_res = np.zeros(glove_model.vector_size)
    ctr = 1
    for w in sent:
        if w in glove_model:
            ctr += 1
            wv_res += glove_model[w]
    wv_res = wv_res/ctr
    #return (wv_res, ctr)
    return wv_res

def get_doc_vecs_glove(X_train,X_test):
    glove_model=loadGlove()
    train_doc_vecs_glove = []
    for doc in X_train:    
        doc_words = [term for term in doc]
        train_doc_vecs_glove.append(sent_vec_glove(doc_words,glove_model))


    test_doc_vecs_glove = []
    for doc in X_test:    
        doc_words = [term for term in doc]
        test_doc_vecs_glove.append(sent_vec_glove(doc_words,glove_model))
    del glove_model
    return train_doc_vecs_glove,test_doc_vecs_glove


In [48]:
#Get TFIDF Score for the given word
def get_tfidf_for_words(text,bpe_vect,tfidf_feature_names):
    tfidf_matrix= bpe_vect.transform([text]).todense()
    feature_index = tfidf_matrix[0,:].nonzero()[1]
    tfidf_scores = zip([tfidf_feature_names[i] for i in feature_index], [tfidf_matrix[0, x] for x in feature_index])
    return dict(tfidf_scores)

In [49]:
#Creation of weighted average sentence vector for BPE
def sent_vec_bpe_domain_weighted(sent,bpemb_en_domain,bpe_vect,tfidf_feature_names):
    wv_res = np.zeros(bpemb_en_domain.emb.vector_size)
    ctr = 0
    test=[]
    wt=get_tfidf_for_words(sent,bpe_vect,tfidf_feature_names)
    #print(wt)
    for w in bpemb_en_domain.encode(sent):
        if w in bpemb_en_domain.emb:
            test.append(wt[w])
            ctr += wt[w]
            wv_res += bpemb_en_domain.emb[w]*wt[w].astype('float32')
    if sum(test)<=0:
        ctr=1
    wv_res = wv_res/ctr
    #print(test,ctr)
    #return (wv_res, ctr)
    wv_res=np.clip(wv_res, a_min = -1, a_max = 1) 
    return wv_res.astype('float32')

def get_doc_vec_bpe_domain_weighted(X_train,X_test):
    bpemb_en_domain=joblib.load("bpemb_en_domain.pkl")
    bpe_vect=TfidfVectorizer(tokenizer=bpemb_en_domain.encode,analyzer="word",vocabulary=bpemb_en_domain.words)
    dfTFIDF=X_train#pd.concat([X_train,X_test])
    dfTFIDF.reset_index(inplace=True,drop=True)
    bpe_vect.fit(dfTFIDF)
    tfidf_feature_names = bpe_vect.get_feature_names()
    
    
    train_doc_vecs_bpe_domain_weighted = []
    for doc in X_train:    
        doc_words = doc
        train_doc_vecs_bpe_domain_weighted.append(sent_vec_bpe_domain_weighted(doc_words,bpemb_en_domain,bpe_vect,tfidf_feature_names))


    test_doc_vecs_bpe_domain_weighted = []
    for doc in X_test:    
        doc_words = doc
        test_doc_vecs_bpe_domain_weighted.append(sent_vec_bpe_domain_weighted(doc_words,bpemb_en_domain,bpe_vect,tfidf_feature_names))
    
    return train_doc_vecs_bpe_domain_weighted,test_doc_vecs_bpe_domain_weighted

In [50]:
#Generating TFIDF transformation for baseline model
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
stop_words=stop_words+['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
stop_words=stop_words+["emailidx",'sap','please','dear','regards','kind','best','help','team','ticket','hi','thank','phonex','thanks','maple','leaf','cc','to','subject','mapleleaf','sent','hello','httpx','email','food','foods']


def getTfIdf(X_train,X_test):
    vect=TfidfVectorizer(stop_words=stop_words,max_features=4096,min_df=5, max_df=0.95)
    vect.fit(X_train)
    X_train_tf=vect.transform(X_train)
    X_test_tf=vect.transform(X_test)
    return X_train_tf,X_test_tf

def getTfIdf_bpe(X_train,X_test):
    bpemb_en_domain=joblib.load("bpemb_en_domain.pkl")
    bpe_vect=TfidfVectorizer(tokenizer=bpemb_en_domain.encode,analyzer="word",vocabulary=bpemb_en_domain.words)
    bpe_vect.fit(X_train)
    X_train_tf_bpe=bpe_vect.transform(X_train)
    X_test_tf_bpe=bpe_vect.transform(X_test)
    return X_train_tf_bpe,X_test_tf_bpe


In [51]:
#Main training loop for the ML model variants for BPE, Glove and TFIDF

logreg = LogisticRegression(penalty="l2", random_state=42, C = 3.5,multi_class="ovr",solver="lbfgs",max_iter=10000)
from sklearn.metrics import f1_score,accuracy_score,precision_score,roc_auc_score,classification_report,recall_score
import tracemalloc,time

def trainModel(X_train,y_train,X_test,y_test,model):
    metric={}
    tracemalloc.start()
    startTime=time.time()
    if model=="glove":
        X_train,X_test=get_doc_vecs_glove(X_train,X_test)
    elif model=="bpe":
        X_train,X_test=get_doc_vecs_bpe(X_train,X_test)
    elif model=="bpe_domain":
        X_train,X_test=get_doc_vecs_bpe_domain(X_train,X_test)
    elif model=="bpe_domain_weighted":
        X_train,X_test=get_doc_vec_bpe_domain_weighted(X_train,X_test)
    elif model=="tfidf":
        X_train,X_test=getTfIdf(X_train,X_test)
    elif model=="tfidf_bpe":
        X_train,X_test=getTfIdf_bpe(X_train,X_test)
    else:
        pass

    logreg.fit(X_train,y_train)
    current, peak = tracemalloc.get_traced_memory()
    #print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
    tracemalloc.stop()
    stopTime=time.time()
    metric["current_memory_usage"]=current/10**6
    metric["peak_memory_usage"]=peak/10**6
    metric["Execution Time"]=stopTime-startTime
    
    pred_train_ys = logreg.predict(X_train)
    pred_test_ys = logreg.predict(X_test)

    metric["Train Accuracy"]=accuracy_score(y_train, pred_train_ys)
    metric["Test Accuracy"]=accuracy_score(y_test, pred_test_ys)
    metric["Precision"]=precision_score(y_test, pred_test_ys,average='weighted')
    metric["F1-Score"]=f1_score(y_test, pred_test_ys,average='weighted')
    metric["Recall"]=recall_score(y_test, pred_test_ys,average='weighted')

    print(model,metric)
    ClassificationReport=classification_report(y_test,pred_test_ys,output_dict=True)
#    print("Train accuracy: ", accuracy_score(y_train, pred_train_ys))
#    print("Test accuracy: ", accuracy_score(y_test, pred_test_ys))
#    print(precision_score(y_test, pred_test_ys,average='weighted'))
#    print(f1_score(y_test, pred_test_ys,average='weighted'))
#    print("Classification Report: \n", classification_report(y_test, pred_test_ys))
    return logreg,metric,ClassificationReport


In [52]:
data={}
d0=pd.DataFrame(columns=['class', 'f1-score', 'precision', 'recall', 'support', 'model'])
for model in ["glove","bpe","bpe_domain","bpe_domain_weighted","tfidf","tfidf_bpe"]:
    LR=trainModel(X_train,y_train,X_test,y_test,model)
    data[model]=LR[1]
    d1=pd.DataFrame(LR[2]).T
    d1["model"]=model
    d1.reset_index(inplace=True)
    d1.columns=['class', 'f1-score', 'precision', 'recall', 'support', 'model']
#,drop=True)
    d0=pd.concat([d0,d1],sort=True)
    d0=d0.reset_index(drop=True)

glove {'current_memory_usage': 14.44514, 'peak_memory_usage': 311.102387, 'Execution Time': 206.32695627212524, 'Train Accuracy': 0.45985003408316294, 'Test Accuracy': 0.34962089300758215, 'Precision': 0.41486706948398705, 'F1-Score': 0.3400736274018411, 'Recall': 0.34962089300758215}
bpe {'current_memory_usage': 14.511728, 'peak_memory_usage': 27.008531, 'Execution Time': 18.22931456565857, 'Train Accuracy': 0.7081799591002045, 'Test Accuracy': 0.5543386689132266, 'Precision': 0.5924936332076558, 'F1-Score': 0.547875722604637, 'Recall': 0.5543386689132266}
bpe_domain {'current_memory_usage': 14.538505, 'peak_memory_usage': 29.311137, 'Execution Time': 19.119107484817505, 'Train Accuracy': 0.7621676891615542, 'Test Accuracy': 0.6149957877000842, 'Precision': 0.642943925809144, 'F1-Score': 0.6028550482207414, 'Recall': 0.6149957877000842}
bpe_domain_weighted {'current_memory_usage': 8.546834, 'peak_memory_usage': 34.151645, 'Execution Time': 60.38460850715637, 'Train Accuracy': 0.742603

In [56]:
#d0.to_csv("dataset2ClassficationReport.csv",index=False)
d0.head()

class  f1-score  model  precision    recall  support
0  Business Support Services  0.377622  glove   0.310345  0.482143     56.0
1             Client Support  0.283186  glove   0.273504  0.293578    109.0
2         HCM System Support  0.272251  glove   0.236364  0.320988     81.0
3              IS Purchasing  0.496000  glove   0.405229  0.639175     97.0
4                     MFGSCP  0.244186  glove   0.205882  0.300000     70.0

In [55]:
#pd.DataFrame(data).T.to_csv("dataset2Result.csv")#,index=False)
pd.DataFrame(data).T

Execution Time  F1-Score  Precision    Recall  \
glove                    206.326956  0.340074   0.414867  0.349621   
bpe                       18.229315  0.547876   0.592494  0.554339   
bpe_domain                19.119107  0.602855   0.642944  0.614996   
bpe_domain_weighted       60.384609  0.562707   0.597077  0.573715   
tfidf                      5.279484  0.765520   0.765415  0.766639   
tfidf_bpe                 14.878838  0.759233   0.759687  0.760741   

                     Test Accuracy  Train Accuracy  current_memory_usage  \
glove                     0.349621        0.459850             14.445140   
bpe                       0.554339        0.708180             14.511728   
bpe_domain                0.614996        0.762168             14.538505   
bpe_domain_weighted       0.573715        0.742604              8.546834   
tfidf                     0.766639        0.979414              6.140200   
tfidf_bpe                 0.760741        0.977232             13.203065   

                     peak_memory_usage  
glove                       311.102387  
bpe                          27.008531  
bpe_domain                   29.311137  
bpe_domain_weighted          34.151645  
tfidf                        13.657696  
tfidf_bpe                    37.967888

### Ticket Dataset already CLEANED of context and sentence structure

In [7]:
pd.read_csv("dataset1Result.csv")# Ticket Dataset already CLEANED of context and sentence structure

Algorithm  Execution Time  F1-Score  Precision  Test Accuracy  \
0                Glove      304.655184  0.500478   0.690032       0.439719   
1                  bpe       56.791766  0.727729   0.801158       0.689153   
2           bpe_domain       48.504760  0.769534   0.823556       0.740097   
3  bpe_domain_weighted      209.438859  0.758605   0.813499       0.728548   
4                tfidf       20.805419  0.847280   0.867641       0.835510   
5            tfidf_bpe       47.851943  0.838732   0.863493       0.825040   
6            CNN_TFIDF      755.337140  0.802285   0.810395       0.819900   

   Train Accuracy  current_memory_usage  peak_memory_usage  
0        0.487112            138.185633         434.856777  
1        0.668137             82.403067         151.742456  
2        0.703445             82.406195         151.746136  
3        0.687261             46.436161         115.776086  
4        0.910369             86.366580         113.204905  
5        0.889248             39.495161          85.674440  
6        0.850200              1.143325          61.163732

### Ticket Dataset RETAINED with context and sentence structure

In [8]:
pd.read_csv("dataset2Result.csv")# Ticket Dataset RETAINED with context and sentence structure

Algorithm  Execution Time  F1-Score  Precision  Test Accuracy  \
0                Glove      205.094482  0.340074   0.414867       0.349621   
1                  bpe       17.846812  0.547876   0.592494       0.554339   
2           bpe_domain       17.540896  0.597670   0.630615       0.609099   
3  bpe_domain_weighted       59.617584  0.564711   0.592736       0.577085   
4                tfidf        6.359000  0.773408   0.774249       0.774221   
5            tfidf_bpe       14.210386  0.759233   0.759687       0.760741   
6            CNN_TFIDF      181.628964  0.679734   0.718331       0.677300   

   Train Accuracy  current_memory_usage  peak_memory_usage  
0        0.459850             14.390190         311.095369  
1        0.708180             14.507995          27.004414  
2        0.758146             14.512475          29.283784  
3        0.738241              8.298961          34.125748  
4        0.987935             10.151310          19.216804  
5        0.977232             13.206081          37.969424  
6        0.782400              1.399523          14.908730